In [1]:
from IPython.core.display import Image, display
display(Image(url='https://www.greekmythology.com/images/mythology/aegis_228.jpg', width=200, unconfined=True))

/tmp/ipykernel_147382/4129434108.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import Image, display


In [2]:
import numpy as np
import healpy as hp
import pickle as pk
import torch
from astropy import units as u
from astropy import constants as c
import matplotlib.pyplot as plt
import seaborn as sns
from os import listdir
import os
import sys
sys.path.append('../astroLFI')
import LFI_galactic_center
from sources import FermiBackgrounds
from sources import Model_O
from sources import DMsignal
from sources import smoothDM
from sources import MSP
from sources import Fermi_Bubbles
from sbi.inference import SNLE, SNPE, prepare_for_sbi, simulate_for_sbi
from sbi import utils as utils
from sbi import analysis as analysis
from sbi.inference.base import infer
from getdist import plots, MCSamples

%matplotlib inline

In [3]:
parameter_range = [[], []]
abundance_luminosity_and_spectrum_list = []
source_class_list = []
parameter_names = []
energy_range = [2000, 100000] #MeV
energy_range_gen = [energy_range[0]*0.5, energy_range[1]*1.5]
luminosity_range = 10.0**np.array([30, 37])
max_radius = 8.5 + 20*2 #kpc
#exposure = 10*2000*10*0.2 #cm^2 yr
exposure = 2000*10*0.2 #cm^2 yr
#exposure = 200*10*0.2 #cm^2 yr
flux_cut = 1e-9 #photons/cm^2/s
#flux_cut = np.inf
angular_cut = 10*u.deg.to('rad') #degrees
angular_cut_gen = angular_cut*1.5
lat_cut = 2*u.deg.to('rad') #degrees
lat_cut_gen = lat_cut*0.5

In [4]:
#parameter_names = ['A_{core}', 'A_{disk}', r'm_{\chi}', r'\langle\sigma_{DM} v\rangle_0', r'A_{O}']
#parameter_range = [[0., 0., 10000., 0.], [0.07, 2., 100000., 10.]] #nonpoisson
parameter_names = ['A_{core}', r'\langle\sigma_{DM} v\rangle_0/m_{\chi}']
#parameter_range = [[0., 0/30000**2], [0.07, 10/30000**2]] #nonpoisson
parameter_range = [[0., 0/30000**2], [3., 10/30000**2]] #poisson

prior = utils.BoxUniform(low = parameter_range[0], high = parameter_range[1])
#prior = utils.BoxUniform(low = parameter_range[0], high = parameter_range[1], device='cuda')

In [5]:
run = 'run9m_poisson'
numsims = '100000sims'

posterior_path = 'posteriors/' + run + '/'
posterior_file = posterior_path + numsims
sim_dir = 'simulations/' + run + '/'
sims = listdir(sim_dir)
theta, x = torch.tensor([]), torch.tensor([])
i = 0
for sim in sims:
    i += 1
    theta_batch, x_batch = np.load(sim_dir + sim, allow_pickle=True)
    theta = torch.cat((theta, theta_batch))
    x = torch.cat((x, x_batch))
    print(i,sim)
    if i == 1000:
        break
inference = SNPE(prior=prior)
#inference = SNPE(prior=prior, device='cuda')
inference = inference.append_simulations(theta, x)
density_estimator = inference.train()
posterior = inference.build_posterior(density_estimator)
os.makedirs(posterior_path, exist_ok=True)
np.save(posterior_file, posterior, allow_pickle=True)

1 sim_249.npy
2 sim_628.npy
3 sim_298.npy
4 sim_467.npy
5 sim_815.npy
6 sim_99.npy
7 sim_48.npy
8 sim_311.npy
9 sim_770.npy
10 sim_592.npy
11 sim_235.npy
12 sim_654.npy
13 sim_122.npy
14 sim_685.npy
15 sim_931.npy
16 sim_543.npy
17 sim_869.npy
18 sim_34.npy
19 sim_232.npy
20 sim_595.npy
21 sim_653.npy
22 sim_682.npy
23 sim_125.npy
24 sim_544.npy
25 sim_936.npy
26 sim_33.npy
27 sim_188.npy
28 sim_159.npy
29 sim_538.npy
30 sim_812.npy
31 sim_460.npy
32 sim_316.npy
33 sim_777.npy
34 sim_938.npy
35 sim_860.npy
36 sim_412.npy
37 sim_364.npy
38 sim_705.npy
39 sim_995.npy
40 sim_240.npy
41 sim_186.npy
42 sim_621.npy
43 sim_157.npy
44 sim_536.npy
45 sim_944.npy
46 sim_291.npy
47 sim_90.npy
48 sim_318.npy
49 sim_41.npy
50 sim_779.npy
51 sim_247.npy
52 sim_992.npy
53 sim_626.npy
54 sim_181.npy
55 sim_150.npy
56 sim_296.npy
57 sim_943.npy
58 sim_531.npy
59 sim_97.npy
60 sim_469.npy
61 sim_46.npy
62 sim_415.npy
63 sim_867.npy
64 sim_363.npy
65 sim_702.npy
66 sim_976.npy
67 sim_504.npy
68 sim_165.n

604 sim_104.npy
605 sim_917.npy
606 sim_565.npy
607 sim_848.npy
608 sim_15.npy
609 sim_899.npy
610 sim_214.npy
611 sim_675.npy
612 sim_103.npy
613 sim_562.npy
614 sim_910.npy
615 sim_780.npy
616 sim_834.npy
617 sim_446.npy
618 sim_330.npy
619 sim_497.npy
620 sim_69.npy
621 sim_751.npy
622 sim_268.npy
623 sim_609.npy
624 sim_112.npy
625 sim_901.npy
626 sim_573.npy
627 sim_205.npy
628 sim_664.npy
629 sim_888.npy
630 sim_859.npy
631 sim_279.npy
632 sim_618.npy
633 sim_78.npy
634 sim_486.npy
635 sim_321.npy
636 sim_740.npy
637 sim_791.npy
638 sim_457.npy
639 sim_825.npy
640 sim_169.npy
641 sim_508.npy
642 sim_326.npy
643 sim_481.npy
644 sim_747.npy
645 sim_796.npy
646 sim_822.npy
647 sim_450.npy
648 sim_115.npy
649 sim_574.npy
650 sim_906.npy
651 sim_202.npy
652 sim_663.npy
653 sim_167.npy
654 sim_506.npy
655 sim_974.npy
656 sim_270.npy
657 sim_611.npy
658 sim_328.npy
659 sim_71.npy
660 sim_749.npy
661 sim_798.npy
662 sim_908.npy
663 sim_881.npy
664 sim_354.npy
665 sim_735.npy
666 sim_850.